In [1]:
import graphlab
import numpy as np
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to sripriya.kuram@gmail.com and will expire on September 13, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\sripr\AppData\Local\Temp\graphlab_server_1477782946.log.0


In [2]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [15]:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') # the [] around 'sqft_living' makes it a list
print example_features[0,:] # this accesses the first row of the data the ':' indicates 'all columns'
print example_output[0] # and the corresponding output

[  1.00000000e+00   1.18000000e+03]
221900.0


In [16]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In [11]:
def feature_derivative(errors, features):
    derivative = 2 * np.dot(errors , features)
    return derivative

In [65]:
from math import sqrt

def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    
    weights = np.array(initial_weights)
    while not converged:
        
        predictions = predict_output(feature_matrix,weights)
        errors =  predictions - output
        gradient_sum_squares = 0
        for i in range(len(weights)):
            a = feature_derivative(errors , feature_matrix[:,i])
            gradient_sum_squares  += (a * a)
            weights[i] = weights[i]  - (step_size * a)
            
        gradient_magnitude = np.sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)
    

In [66]:
train_data,test_data = sales.random_split(.8,seed=0)

In [67]:
simple_features = ['sqft_living']
my_output= 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
#x=0
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [68]:
simple_weights = regression_gradient_descent(simple_feature_matrix, output,initial_weights, step_size,tolerance)
print simple_weights

[-46999.88716555    281.91211912]


In [75]:
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)
pred = predict_output(test_simple_feature_matrix,simple_weights)
test_simple_weights = regression_gradient_descent(test_simple_feature_matrix, output,initial_weights, step_size,tolerance)
print pred[0]

356134.443171


In [84]:
def rss(predictions , output):
    rss=0
    r = output - predictions
    rss= r*r
    rss = sum(rss)
    return rss

In [85]:
rss(pred , test_output)

275400047593155.69

In [86]:
model_features = ['sqft_living', 'sqft_living15']
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features,my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [87]:
feature_weights = regression_gradient_descent(feature_matrix, output,initial_weights, step_size,tolerance)
print feature_weights

[ -9.99999688e+04   2.45072603e+02   6.52795277e+01]


In [95]:

test_feature_matrix, test_output2 = get_numpy_data(test_data, model_features,my_output)
pred2 = predict_output(test_feature_matrix, feature_weights)
print pred2[0]

366651.412037


In [96]:
print test_output2[0]

310000.0


In [97]:
rss(pred2 , test_output2)

270263446465243.97